# data preparation

In [2]:
# iris dataset
# https://archive.ics.uci.edu/dataset/53/iris

import urllib.request
urllib.request.urlretrieve('https://archive.ics.uci.edu/static/public/53/iris.zip',
                          'data.zip')

('data.zip', <http.client.HTTPMessage at 0x7f61141b2170>)

In [3]:
# unzip downloaded data
!mkdir data
!unzip data.zip -d data

Archive:  data.zip
  inflating: data/Index              
  inflating: data/bezdekIris.data    
  inflating: data/iris.data          
  inflating: data/iris.names         


In [4]:
import pandas as pd

# read data
data =pd.read_csv('data/iris.data', header=None)

print(data)

       0    1    2    3               4
0    5.1  3.5  1.4  0.2     Iris-setosa
1    4.9  3.0  1.4  0.2     Iris-setosa
2    4.7  3.2  1.3  0.2     Iris-setosa
3    4.6  3.1  1.5  0.2     Iris-setosa
4    5.0  3.6  1.4  0.2     Iris-setosa
..   ...  ...  ...  ...             ...
145  6.7  3.0  5.2  2.3  Iris-virginica
146  6.3  2.5  5.0  1.9  Iris-virginica
147  6.5  3.0  5.2  2.0  Iris-virginica
148  6.2  3.4  5.4  2.3  Iris-virginica
149  5.9  3.0  5.1  1.8  Iris-virginica

[150 rows x 5 columns]


In [5]:
# convert to numerical values
data[4] = data[4].replace('Iris-setosa', 0)
data[4] = data[4].replace('Iris-virginica', 1)
data[4] = data[4].replace('Iris-versicolor', 2)

print(data)

       0    1    2    3  4
0    5.1  3.5  1.4  0.2  0
1    4.9  3.0  1.4  0.2  0
2    4.7  3.2  1.3  0.2  0
3    4.6  3.1  1.5  0.2  0
4    5.0  3.6  1.4  0.2  0
..   ...  ...  ...  ... ..
145  6.7  3.0  5.2  2.3  1
146  6.3  2.5  5.0  1.9  1
147  6.5  3.0  5.2  2.0  1
148  6.2  3.4  5.4  2.3  1
149  5.9  3.0  5.1  1.8  1

[150 rows x 5 columns]


In [6]:
# shuffle data
data = data.sample(frac=1).reset_index(drop=True)

print(data)

       0    1    2    3  4
0    6.1  2.8  4.0  1.3  2
1    6.7  3.1  4.7  1.5  2
2    5.6  3.0  4.5  1.5  2
3    5.5  2.3  4.0  1.3  2
4    5.6  2.5  3.9  1.1  2
..   ...  ...  ...  ... ..
145  6.9  3.1  5.4  2.1  1
146  5.0  2.0  3.5  1.0  2
147  6.4  2.9  4.3  1.3  2
148  5.1  2.5  3.0  1.1  2
149  6.8  2.8  4.8  1.4  2

[150 rows x 5 columns]


In [7]:
# change the label column index
data = data[[4, 0, 1, 2, 3]]

print(data)

     4    0    1    2    3
0    2  6.1  2.8  4.0  1.3
1    2  6.7  3.1  4.7  1.5
2    2  5.6  3.0  4.5  1.5
3    2  5.5  2.3  4.0  1.3
4    2  5.6  2.5  3.9  1.1
..  ..  ...  ...  ...  ...
145  1  6.9  3.1  5.4  2.1
146  2  5.0  2.0  3.5  1.0
147  2  6.4  2.9  4.3  1.3
148  2  5.1  2.5  3.0  1.1
149  2  6.8  2.8  4.8  1.4

[150 rows x 5 columns]


In [9]:
# split into train and validation set
train_data = data[:120]
val_data = data[120:]

# move data into s3 packet

In [11]:
import boto3

bucket_name = 'sagemaker-build-and-deploy1'

train_data.to_csv('data.csv', header=False, index=False)
key = 'data/train/data'
url = 's3://{}/{}'.format(bucket_name, key)
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('data.csv')

val_data.to_csv('data.csv', header=False, index=False)
key = 'data/val/data'
url = 's3://{}/{}'.format(bucket_name, key)
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('data.csv')

# create model

In [15]:
#create object to train the model

import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import get_execution_role

key = 'model/xgb_model'
s3_output_location = 's3://{}/{}'.format(bucket_name, key)

xgb_model = sagemaker.estimator.Estimator(
    get_image_uri(boto3.Session().region_name, 'xgboost'),
    get_execution_role(),
    train_instance_count=1,
    train_instance_type='ml.m4.xlarge',
    train_volume_size=5,
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session()
)

#aws documentation for xgboost hyperparameters
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
xgb_model.set_hyperparameters(max_depth=5,
                             eta=0.2,
                             gamma=4,
                             min_child_weight=6,
                             silent=0,
                             objective='multi:softmax',
                             num_class=3,
                             num_round=10)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


# train model

In [16]:
#refering to the directory containing the data
train_data = 's3://{}/{}'.format(bucket_name, 'data/train')
val_data = 's3://{}/{}'.format(bucket_name, 'data/val')

train_channel = sagemaker.session.s3_input(train_data, content_type='text/csv')
val_channel = sagemaker.session.s3_input(val_data, content_type='text/csv')

data_channels = {'train': train_channel, 'validation': val_channel}

xgb_model.fit(inputs=data_channels)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Creating training-job with name: xgboost-2024-10-26-09-14-38-448


2024-10-26 09:14:39 Starting - Starting the training job......
2024-10-26 09:15:15 Starting - Preparing the instances for training......
2024-10-26 09:16:30 Downloading - Downloading input data...
2024-10-26 09:17:11 Downloading - Downloading the training image......
2024-10-26 09:18:06 Training - Training image download completed. Training in progress...Arguments: train
[2024-10-26:09:18:17:INFO] Running standalone xgboost training.
[2024-10-26:09:18:17:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 8455.31mb
[2024-10-26:09:18:17:INFO] Determined delimiter of CSV input is ','
[09:18:17] S3DistributionType set as FullyReplicated
[09:18:17] 120x4 matrix with 480 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-10-26:09:18:17:INFO] Determined delimiter of CSV input is ','
[09:18:17] S3DistributionType set as FullyReplicated
[09:18:17] 30x4 matrix with 120 entries loaded from /opt/ml/input/data/validation

# deploy model 

In [17]:
xgb_predictor = xgb_model.deploy(initial_instance_count=1,
                                instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2024-10-26-09-30-44-780
INFO:sagemaker:Creating endpoint-config with name xgboost-2024-10-26-09-30-44-780
INFO:sagemaker:Creating endpoint with name xgboost-2024-10-26-09-30-44-780


------!